In [2]:
import re
import os
import time
import pickle
import requests
from zoneinfo import ZoneInfo
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow as iaf

os.chdir('/Users/ettore/Desktop/')

In [3]:
## API setup

scope       = ['https://www.googleapis.com/auth/calendar']
flow        = iaf.from_client_secrets_file('client_secret.json', scopes=scope)
credtype    = 'flow' # can be 'flow' or 'pickle'

# pickle.dump(credentials, open('token.pkl', 'wb'))

if credtype == 'flow':
    credentials = flow.run_console()
elif credtype == 'pickle':
    try:
        credentials = pickle.load(open('token.pkl', 'rb'))
    except:
        print('Pickle not found')
else:
    print('Please specify credtype correctly')
    
service = build('calendar', 'v3', credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=986027283668-umflbqc7g1s7alpv4nrgmqccilu4gev1.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=RNiFSw5AHccCK1AB4VX29JlYhbpBTr&prompt=consent&access_type=offline


Enter the authorization code:  4/1AX4XfWgyfSAA8hPUlO37uwlQ0kgnyEFiEAyIYETrpDR4DUQjJ1PLIlUoVtw


In [4]:
## Scraping

URL  = "https://www.lab111.nl/programma/listview/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "lxml")
forecast = 3 # +1

program = []
for day in range(forecast):
    movielist = soup.find_all('tr', class_=f'day{day}')
    movielist = movielist[1:]
    
    for movie in movielist: 
        movie_url = re.findall('(?P<url>https?://[^\s]+")',str(movie))[1]
        movie_url = movie_url[:-1]        
        
        shows = dict()
        shows ={
          "s_time": movie.findAll('a')[0].text,
          "duration": BeautifulSoup(requests.get(movie_url).content, 'lxml').find_all('ul', class_="speelduur")[0].text,
          "name": movie.findAll('a')[1].text,
          "day": day,
          "lab": movie.find('span').text,
          "ticket": str(movie.find('a', class_='button tic')).split('\"')[3]
        }
        program.append(shows)

In [6]:
def check_lab(result):
    check = False
    for i in range(len(result['items'])):
        if result['items'][i]['summary'] == 'lab111':
            check = True
    return check

calendars = service.calendarList().list().execute()
if not check_lab(calendars):
    calendar = {
    'summary': 'lab111',
    'timeZone': 'Europe/Amsterdam',
    }
    created_calendar = service.calendars().insert(body=calendar).execute()
    print('Calendar created')
else:
    print('Calendar not created')

Calendar not created


In [7]:
def create_event(show):
    
    now      = datetime.now()
    tz       = ZoneInfo("Europe/Amsterdam").key
    starT    = [int(x) for x in show['s_time'].split(':')] # hour and minutes
    duration = [int(x) for x in re.findall(r'\d+', show['duration'])]
    startTime= datetime(now.year, now.month, (now.day + show['day']), starT[0], starT[1])
    endTime  = startTime + timedelta(hours=duration[0], minutes=duration[1])
    
    event    = {
      'summary': show['name'],
      'location': show['lab'],
      'description': show['ticket'],
      'start': {
        'dateTime': startTime.isoformat(),
        'timeZone': tz,
      },
      'end': { 
        'dateTime': endTime.isoformat(),
        'timeZone': tz,
      }
    }
    return event

In [9]:
# get calendard id of lab111 calendar
out_cal = 0
for cal in calendars['items']:
    if cal['summary'] == 'lab111':
        calendarId = cal['id']
        
# get lab events already present
cal_events = []
page_token = None
while True:
  events = service.events().list(calendarId=calendarId, pageToken=page_token).execute()
  for event in events['items']:
    cal_events.append(str(event['description']))
  page_token = events.get('nextPageToken')
  if not page_token:
    break    

# add events
counts = [0, 0]
for show in program:
    ticket_id = show['ticket']
    if not ticket_id in cal_events:
        service.events().insert(calendarId = calendarId, body = create_event(show)).execute()
        counts[0] += 1
        time.sleep(.3)
    else:
        counts[1] += 1
print(f'{counts[0]} shows were added and {counts[1]} shows were already present')

0 shows were added and 41 shows were already present
